In [78]:
import numpy as np
import pandas as pd
import json
import requests
cities=pd.read_csv('dests.txt', header=None)
api_key='*****'
cities

,0
0,Istanbul
1,Amsterdam
2,Valletta
3,Basel
4,Doha


In [74]:
data = pd.DataFrame(columns=['Target', 'Distance_km', 'Duration (hour+minutes)', 'Longitude', 'Latitude'])
for des in cities.values:
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (des,api_key)
    response = requests.get(url)
    res=response.json()
    lat=res['results'][0]['geometry']['location']['lat']
    lng=res['results'][0]['geometry']['location']['lng']
    url_dis="https://maps.googleapis.com/maps/api/distancematrix/json?origins=%s&destinations=%s&key=%s" % ('Tel Aviv',des,api_key)
    response_dis = requests.get(url_dis)
    res_dis=response_dis.json()
    distance=res_dis['rows'][0]['elements'][0]['distance']['value']/1000
    duration=int(res_dis['rows'][0]['elements'][0]['duration']['value']/60)
    new_row = {'Target': des[0], 'Distance_km': distance, 'Duration (hour+minutes)': duration, 'Longitude': lng, 'Latitude': lat}
    data = data.append(new_row,ignore_index=True)

In [75]:
data

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
0,Istanbul,1815.227,1262,28.978359,41.008238
1,Amsterdam,4532.541,2883,4.904139,52.367573
2,Valletta,3792.883,3051,14.514100,35.899237
3,Basel,4093.629,2647,7.588576,47.559599
4,Doha,2164.477,1358,51.531040,25.285447


In [76]:
three_furthest=data.nlargest(3,'Distance_km')
three_furthest

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
1,Amsterdam,4532.541,2883,4.904139,52.367573
3,Basel,4093.629,2647,7.588576,47.559599
2,Valletta,3792.883,3051,14.514100,35.899237
